### The sample of text extraction (only 1 pdf)

In [1]:
import torch
import base64
import os
from io import BytesIO
from PIL import Image
from transformers import AutoProcessor, Qwen2VLForConditionalGeneration

from olmocr.data.renderpdf import render_pdf_to_base64png
from olmocr.prompts import build_finetuning_prompt
from olmocr.prompts.anchor import get_anchor_text

# Initialize the model
model = Qwen2VLForConditionalGeneration.from_pretrained("allenai/olmOCR-7B-0225-preview", torch_dtype=torch.bfloat16).eval()
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct",use_fast=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Grab a sample PDF
# urllib.request.urlretrieve("https://molmo.allenai.org/paper.pdf", "./paper.pdf")

# Render page 1 to an image
image_base64 = render_pdf_to_base64png("C:/Users/Tommy/Downloads/ESGreporttemplate/A/2023-CSR-report_e.pdf", 6, target_longest_image_dim=1024)

# Build the prompt, using document metadata
# anchor_text = get_anchor_text("C:/Users/Tommy/Downloads/ESGreporttemplate/A/2023-CSR-report_e.pdf", 5, pdf_engine="pdfreport", target_length=4000)
# prompt = build_finetuning_prompt(anchor_text)

#print(prompt)
# Build the full prompt
messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "Below is the image of one page of a document, as well as some raw textual content that was previously extracted for it. Just return the plain text representation of this document as if you were reading it naturally. Do not hallucinate. output **only** the natural text content."},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_base64}"}},
                ],
            }
        ]

# Apply the chat template and processor
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
main_image = Image.open(BytesIO(base64.b64decode(image_base64)))

inputs = processor(
    text=[text],
    images=[main_image],
    padding=True,
    return_tensors="pt",
)
inputs = {key: value.to(device) for (key, value) in inputs.items()}


# Generate the output
output = model.generate(
            **inputs,
            temperature=0.8,
            max_new_tokens=4000,
            num_return_sequences=1,
            do_sample=True,
        )

# Decode the output
prompt_length = inputs["input_ids"].shape[1]
new_tokens = output[:, prompt_length:]
text_output = processor.tokenizer.batch_decode(
    new_tokens,
    skip_special_tokens=True
)

print(text_output)
# ['{"primary_language":"en","is_rotation_valid":true,"rotation_correction":0,"is_table":false,"is_diagram":false,"natural_text":"Molmo and PixMo:\\nOpen Weights and Open Data\\nfor State-of-the']


C:\Users\Tommy\anaconda3\envs\MultiModel_Analysis\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 14.54it/s]
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


['{"primary_language":"en","is_rotation_valid":true,"rotation_correction":0,"is_table":false,"is_diagram":false,"natural_text":"About Taishin\\n\\nCompany Profile\\n\\nBasic Information\\nEstablished in 2002, Taishin FHC is committed to leading its subsidiaries and related companies with innovative business models based on the core values of \\"Integrity, Commitment, Innovation, and Collaboration\\" to continue to provide more comprehensive services and innovative products. With high-quality services, we have earned the recognition and trust of society, worked side-by-side with our clients, and become their \\"Smart Partner\\". We aim to become a consistent and growing trustworthy financial institution, a role model in corporate governance and social responsibilities, a leader of Taiwan’s financial industry, and a leading brand of the Chinese community.\\n\\nGlobal Operation Strategy\\nIn order to establish a complete financial network, accelerate the internationalization of business g

### Extract the natural text from 1 page pdf

In [2]:
import json

output_data = json.loads(text_output[0])
print(output_data['natural_text'])
# !python -m olmocr.pipeline ./localworkspace --markdown --pdfs D:\pycharmProjects\File-Multimodel-Analysis\Data\PDF\2023-CSR-report_e.pdf

About Taishin

Company Profile

Basic Information
Established in 2002, Taishin FHC is committed to leading its subsidiaries and related companies with innovative business models based on the core values of "Integrity, Commitment, Innovation, and Collaboration" to continue to provide more comprehensive services and innovative products. With high-quality services, we have earned the recognition and trust of society, worked side-by-side with our clients, and become their "Smart Partner". We aim to become a consistent and growing trustworthy financial institution, a role model in corporate governance and social responsibilities, a leader of Taiwan’s financial industry, and a leading brand of the Chinese community.

Global Operation Strategy
In order to establish a complete financial network, accelerate the internationalization of business goals, and strengthen its competitiveness, Taishin FHC completed the merger of Prudential Life Insurance on June 30, 2021 and the name changed to Taishin